In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations
from collections import Counter, defaultdict
from astropy.table import Table, join, vstack
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord
from easyquery import Query, QueryMaker
from astropy.io import ascii

import SAGA
from SAGA.database import FitsTable, GoogleSheets
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord
from SAGA.utils.distance import z2v, d2m, m2d
from SAGA.utils.display import show_images
from SAGA.objects.object_catalog import calc_fiducial_p_sat, calc_fiducial_p_sat_corrected
print(SAGA.__version__)

from scipy.stats import norm

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_all.fits").read()
sats = Query(C.is_sat, C.r_abs_limit, saga.host_catalog.construct_host_query("paper2_complete")).filter(base)
del base

In [ ]:
assert len(sats) == 123
assert len(np.unique(sats["HOSTID"])) == 33

In [ ]:
rbins = np.linspace(10, 300, 1001)

In [ ]:
def get_cdf(x, bins):
    return np.searchsorted(np.sort(x), bins) / len(x)

def print_range(arr):
    a = np.round(arr).astype(np.int)
    print("{}^{{+{}}}_{{-{}}}".format(a[1], a[2]-a[1], a[1]-a[0]))
    print()

In [ ]:
saga_median_r = np.array([np.median(sats_this['RHOST_KPC']) for sats_this in sats.group_by("HOSTID").groups])
print_range(np.quantile(saga_median_r, norm.sf([1, 0, -1])))

mask1 = saga_median_r < np.quantile(saga_median_r, 0.25)
mask2 = saga_median_r > np.quantile(saga_median_r, 0.75)

saga_radial = np.vstack([get_cdf(sats_this['RHOST_KPC'], rbins) for sats_this in sats.group_by("HOSTID").groups])
saga_radial_mean = saga_radial.mean(axis=0)
saga_radial_std = saga_radial.std(axis=0)
saga_radial_pct = np.quantile(saga_radial, norm.sf([1, 0, -1]), axis=0)

saga_radial_mean1 = saga_radial[mask1].mean(axis=0)
saga_radial_mean2 = saga_radial[mask2].mean(axis=0)

In [ ]:
dhalf = [np.median(Query("RHOST_KPC < 150").filter(sats_this, "RHOST_KPC")) for sats_this in sats.group_by("HOSTID").groups if Query("RHOST_KPC < 150").count(sats_this)]
print_range(np.quantile(dhalf, norm.sf([1, 0, -1])))

plt.hist(dhalf, np.linspace(10, 150, 6), alpha=0.6)
plt.xlim(0, 160)
plt.ylim(0, 10)
plt.scatter(dhalf, np.ones_like(dhalf)*9.5, s=5)
plt.xlabel("SAGA $d_\\mathrm{proj, half}$ (limited to sats $< 150$ kpc)");

In [ ]:
from scipy.integrate import quad, cumtrapz
from math import sqrt

def g(c):
    return 1.0 / (np.log1p(c) - c / (1 + c))

def Cinv(x):
    with np.errstate(invalid="ignore"):
        return np.where(x > 1, np.arccosh(x),  np.arccos(x))

def nfw_proj(x, c):
    y = c*x
    return g(c) * (Cinv(1/y) / np.sqrt(np.abs(y*y-1)) + np.log(y/2))

def nfw_rho(x, c=3):
    y = c*x
    return 1/y/(1+y)**2

def Sigma(R, rho_func, r_lim=1):
    return quad(lambda r: r * rho_func(r) / sqrt(r + R), R, r_lim, weight="alg", wvar=(-0.5, 0))[0]

def uniform(x, lim_inner=10, lim_outer=300):
    ao = lim_outer*lim_outer
    ai = lim_inner*lim_inner
    return (x*x - ai) / (ao-ai)

def get_theory(rho_func):
    Rall = np.linspace(10, 300, 10001)[:-1] / 300
    S = np.fromiter((Sigma(R, rho_func, r_lim=1) for R in Rall), dtype=np.float, count=len(Rall))
    S_int = cumtrapz(S*Rall, Rall, initial=0)
    S_int /= S_int[-1]
    return Rall*300, S_int

In [ ]:
r_th, s_th = get_theory(lambda r: 1/r)

In [ ]:
lg = SAGA.database.FitsTable("../data/McConnachie2012.fit").read()
lg["Mr"] = lg["vmag_lc"] - 0.4
lg = (Query((np.isfinite, "vmag_lc"), ~QueryMaker.equals("Name", "Canis Major"), "Mr < -12.295") | QueryMaker.equals("Name", "Andromeda")).filter(lg)
lg["Mr"][lg["Name"] == "Andromeda"] = -22

In [ ]:
xyz = SkyCoord(lg["RAJ2000"], lg["DEJ2000"], lg["D"], unit=("deg", "deg", "kpc")).galactocentric.cartesian.xyz.value.T
xyz = np.vstack([xyz, [0, 0, 0]])

In [ ]:
idx_m31 = np.argmax(lg["Name"] == "Andromeda")
idx_mw = len(lg)

In [ ]:
def generate_projections(xyz, idx_host, idx_other, z_limit=500, n_needed=10):
    realization = []
    xyz_shifted = xyz - xyz[idx_host]
    while len(realization) < n_needed:
        m_rand = np.linalg.qr(np.random.randn(3, 3))[0]
        xyz_rot = np.dot(xyz_shifted, m_rand)
        r_proj = np.hypot(xyz_rot[:,0], xyz_rot[:,1])
        if r_proj[idx_other] < 300:
            continue
        realization.append(r_proj[(r_proj > 10) & (r_proj < 300) & (np.abs(xyz_rot[:,2]) < z_limit)])
    return realization

In [ ]:
mw_realization = generate_projections(xyz, idx_mw, idx_m31, n_needed=5000)

In [ ]:
m31_realization = generate_projections(xyz, idx_m31, idx_mw, n_needed=5000)

In [ ]:
mw_radial = np.vstack([get_cdf(r, rbins) for r in mw_realization])
m31_radial = np.vstack([get_cdf(r, rbins) for r in m31_realization])

In [ ]:
mw_radial_mean = mw_radial.mean(axis=0)
mw_radial_std = mw_radial.std(axis=0)
mw_radial_pct = np.quantile(mw_radial, norm.sf([1, 0, -1]), axis=0)

m31_radial_mean = m31_radial.mean(axis=0)
m31_radial_std = m31_radial.std(axis=0)
m31_radial_pct = np.quantile(m31_radial, norm.sf([1, 0, -1]), axis=0)

In [ ]:
plt.figure(figsize=(4.8, 3.8))

plt.plot(rbins, mw_radial_pct[1], c="C1", label='MW', lw=1, ls="--", zorder=20);
plt.plot(rbins, m31_radial_pct[1], c="C7", label='M31', lw=1, ls="-.", zorder=10);

plt.fill_between(rbins, mw_radial_pct[0], mw_radial_pct[2], color="C1", alpha=0.25, lw=0, zorder=5, rasterized=True);
plt.fill_between(rbins, m31_radial_pct[0], m31_radial_pct[2], color="C7", alpha=0.25, lw=0, zorder=1, rasterized=True);


plt.plot(rbins, saga_radial_mean, c="C2", label='SAGA', lw=2, zorder=50)
plt.plot(rbins, saga_radial_mean1, c="C2", label='SAGA 25%\nmost conc.', lw=0.8, zorder=40);

plt.plot(r_th, s_th, c="C0", label='$\\rho(r) = r^{-1}$', ls=":", zorder=30);


(lines, labels) = plt.gca().get_legend_handles_labels()
l = plt.Line2D([0], [0], linestyle='none', marker='')
for _  in range(2):
    lines.insert(0, l)
    labels.insert(0, "")

plt.axhline(0.0, c='k', ls=':', lw=0.5, zorder=0);
plt.axhline(1.0, c='k', ls=':', lw=0.5, zorder=0);
plt.axvline(10.0, c='k', ls=':', lw=0.5, zorder=0);
plt.axvline(300.0, c='k', ls=':', lw=0.5, zorder=0);
plt.legend(lines, labels, loc="lower right", labelspacing=0.3, fontsize=13, handlelength=1, handletextpad=0.5,ncol=2, columnspacing=1)
plt.xlabel('$d_\\mathrm{proj}$ [kpc]');
plt.ylabel('$N(<d_\\mathrm{proj}) \, / \, N(< 300\,\\mathrm{kpc})$');
plt.tight_layout()
plt.savefig('/home/yymao/Downloads/sat_radial.pdf', bbox_inches='tight', dpi=200)